## Импорт и определение библиотек

In [2]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# ставим разные пакеты оттуда
!pip install langchain langchain-openai langchain-community openai tiktoken langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu unstructured pypandoc -q
#huggingface_hub  langchain_experimental langchainhub
#!pip install unstructured "unstructured[pdf]" -q
!pip install sentence-transformers numpy transformers requests -q

In [4]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
import tqdm as notebook_tqdm
from huggingface_hub import HfApi

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [5]:
# Чтение токена из файла
def read_token(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
hf_token = read_token('../utils.key')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    #"cointegrated/LaBSE-en-ru",
    model_kwargs={
        "device": "cpu",
        "token": hf_token
                  } 
)

/tmp/ipykernel_239209/347647269.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(


## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [6]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub))
print('max chars on page:', max([len(page.page_content) for page in doc_epub]))

pages count: 1
max chars on page: 107261


In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

### Исполним `Text splitters` и разобьём на чанки

In [8]:
from langchain.text_splitter import TextSplitter
from transformers import AutoTokenizer

class TransformersTokenSplitter(TextSplitter):
    """Кастомный сплиттер для токенизаторов Hugging Face."""
    
    def __init__(self, tokenizer, chunk_size=256, chunk_overlap=64):
        super().__init__(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        self.tokenizer = tokenizer

    def split_text(self, text: str) -> list[str]:
        # Токенизируем текст
        tokens = self.tokenizer.encode(text, add_special_tokens=False)
        
        # Разбиваем токены на чанки
        chunks = []
        current_chunk = []
        current_length = 0
        
        for token in tokens:
            current_chunk.append(token)
            current_length += 1
            
            if current_length >= self._chunk_size:
                # Декодируем чанк обратно в текст
                chunk_text = self.tokenizer.decode(current_chunk)
                chunks.append(chunk_text)
                
                # Сохраняем перекрытие
                current_chunk = current_chunk[-self._chunk_overlap :]
                current_length = len(current_chunk)
        
        # Добавляем последний чанк
        if current_chunk:
            chunk_text = self.tokenizer.decode(current_chunk)
            chunks.append(chunk_text)
            
        return chunks

# -----------------------------------------------------------
# Пример использования
# -----------------------------------------------------------

# Загрузка токенизатора DeepSeek
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")

# Инициализация сплиттера
splitter = TransformersTokenSplitter(
    tokenizer=tokenizer,
    chunk_size=256,      # Размер чанка в токенах
    chunk_overlap=64     # Перекрытие между чанками
)

Получено 1 чанков.


In [9]:
# Разделяем книгу на части
#documents = splitter.split_documents(doc_epub)

split_documents = []
for page in doc_epub:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(split_documents)

Token indices sequence length is longer than the specified maximum sequence length for this model (48723 > 4096). Running this sequence through the model will result in indexing errors


254

In [10]:
chunks = splitter.split_documents(doc_epub)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

Количество чанков: 254
Максимальное количество символов в чанке: 707


### Наполним чашу `Vector Store`

In [11]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub")

## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [12]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [17]:
"""query = "грейпфрутовый сок, ржаной виски"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)"""

РЦИЙ:

350 мл ржаного виски;

120 мл ананасового сока;

60 мл лимонного сока;

1 л имбирного пива (истинно американский напиток, который непросто отыскать на наших просторах).

Виски и соки заливаем в чашу для пунша, где уже лежит глыба льда. Вливаем имбирное пиво, трубим общий сбор – пора развеять экзистенциальную тоску!

Приключения Шербета Холмса

ПРИКЛЮЧЕНИЯ ШЕРЛОКА ХОЛМСА (1891–1892)

СЭР АРТУР КОНАН ДОЙЛЬ

Да будет вам известно: дословно фразу «Элементарно, Ватсон» Шерлок Холмс в книгах не произносит.
 земляники;

8 свежих тщательно промытых веточек мяты;

15 мл лимонного сока;

30 мл сиропа агавы;

50 мл светлого рома;

сильногазированная вода для долива.

В стакане коллинз толчем мадлером ягоды, мяту, сок и сироп. Добавляем две пригоршни льда и ром, тщательно перемешиваем, доверху доливаем газировкой. Замираем в ожидании, когда прольются слезы радости.

Лайм для старого морехода

СКАЗАНИЕ О СТАРОМ МОРЕХОДЕ (1798)

СЭМЮЭЛ ТЕЙЛОР КОЛЬРИДЖ

Когда в следующий раз застрянете
ки лимо

# Месье системный промт

In [18]:
# Системный промт для агента от DeepSeek

system_prompt = """Ты — помощник для поиска рецептов коктейлей. Используй только данные из контекста ниже:
{context}

Инструкции:
1. Название коктейля
2. Ингредиенты с объемами
3. Шаги приготовления
"""

# Промпт с обязательными переменными {context} и {question}
qa_prompt = PromptTemplate(
    template=system_prompt + "\nВопрос: {question}\nОтвет:",
    input_variables=["context", "question"]  # Явно указываем обе переменные
)

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [ ]:
# Инициализация модели DeepSeek
llm = HuggingFaceEndpoint(
        repo_id="deepseek-ai/deepseek-llm-7b-chat",         
        task="text-generation", 
        max_new_tokens=1024,
        temperature=0.3,
        top_p=0.95,
        repetition_penalty=1.1,
        huggingfacehub_api_token=hf_token
    ) 


In [37]:

# объявление retriever 
retriever = db_embed.as_retriever(
        search_type="mmr",              # тип поиска похожих документов
        search_kwargs={
                "k": 5,                 # количество возвращаемых документов (Default: 4)
                "fetch_k": 20
            },     
    )


# модификация цепочки RetrievalQA
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": qa_prompt},  # Используем исправленный шаблон
    return_source_documents=True
)

# Миксуем рецепт коктейля `agent` 00х

In [38]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [39]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "по-умолчанию")


In [40]:
def main():
    # Определение стилей
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "по-умолчанию"  # Стиль по умолчанию
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]
        # Выбор стиля
        style = choose_style(styles)
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt) #.content
        
        # Переформулировка заголовков
        styled_response = styled_response.replace("Коктейль:", "Название коктейля:") 
        styled_response = styled_response.replace("Ингредиенты:", "Состав:") 
        styled_response = styled_response.replace("Приготовление:", "Способ приготовления:")
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()


HfHubHTTPError: (Request ID: PdsyMXSmMfQs2P7fpb1A6)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/IlyaGusev/saiga_llama3_8b.
Make sure your token has the correct permissions.
The model IlyaGusev/saiga_llama3_8b is too large to be loaded automatically (16GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

## Теперь озвучка

In [89]:
# импортируем библиотеки
import requests
import uuid

# функция для получения токена
def get_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «SALUTE_SPEECH_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return None

In [90]:
# Считываем ключ авторизации из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('SBER_API_KEY'):
            SBER_API_KEY = line.split('=', 1)[1].strip(' \n\r')
            if not SBER_API_KEY.endswith('=='):  # Если нет ==, добавляем их
                SBER_API_KEY += '=='
            break

print(f"Прочитанное значение: '{SBER_API_KEY}'")
#'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='

response = get_token(SBER_API_KEY) # получаем токен
if response != None:
    salute_token = response.json()['access_token']

Прочитанное значение: 'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='


In [92]:
# функция для синтеза речи
def synthesize_speech(text, token, format='wav16', voice='May_24000'):
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize" # URL для синтеза речи
    headers = {
        "Authorization": f"Bearer {token}", # токен авторизации
        "Content-Type": "application/text" # тип содержимого
    }
    params = {
        "format": format, # формат аудио
        "voice": voice # голос
    }
    response = requests.post(
            url, 
            headers=headers, 
            params=params, 
            data=text.encode(), # текст для синтеза
            verify=False # отключение проверки SSL
        )

    # проверка на успешность синтеза
    if response.status_code == 200:
        # Сохранение синтезированного аудио в файл
        with open('output.wav', 'wb') as f:
            f.write(response.content)
        print("Аудио успешно синтезировано и сохранено как 'output.wav'")

In [102]:
# вызов функции для синтеза речи
synthesize_speech(recipe, salute_token)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'


In [32]:
"""## Если не хотите платить денежков, то запустите эту ячейку
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# Создаем объект языкового модели
llm = HuggingFaceEndpoint(
      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
      huggingfacehub_api_token = hf_token
    )"""